In [58]:
import numpy as np
import pandas as pd
import time
import tkinter as tk
from openpyxl import load_workbook
import undetected_chromedriver as uc
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
from tabulate import tabulate
from webdriver_manager.chrome import ChromeDriverManager
servico = Service(ChromeDriverManager().install())
import funcoes as fc

In [59]:
#INICIA O NAVEGADOR
navegador = uc.Chrome(service=servico)
navegador.maximize_window()
navegador.get('https://sei.anatel.gov.br/')


In [74]:
lista_procConformes = ["53500.074124/2024-32", "53500.073842/2024-91"]
lista_processos = []
processo = ["53500.077392/2024-14"]

In [75]:
# FUNÇÃO PARA VERIFICAR A EXISTÊNCIA DE ELEMENTOS NA TELA UTILIZANDO O SELENIUM
def check_element_exists(by, value, navegador):
    # Opcional: aguardar um pouco antes de verificar
    time.sleep(0.5)  # Espera de meio segundo antes da verificação
    try:
        navegador.find_element(by, value)
        return True
    except NoSuchElementException:
        #print(f"Elemento não encontrado: {value}")  # Log de erro
        return False

In [76]:
janela_principal = navegador.current_window_handle
navegador.switch_to.default_content()
navegador.switch_to.frame('ifrArvore')
if check_element_exists(By.PARTIAL_LINK_TEXT, "Declaração de Conformidade - Drone", navegador):
    tipo_processo = "Drone"
elif check_element_exists(By.PARTIAL_LINK_TEXT, "Declaração de Conformidade - Importado Uso Próprio", navegador):
    tipo_processo = "Importado"

tipo_processo

'Importado'

In [81]:
janela_principal = navegador.current_window_handle
#PESQUISA NUMERO DO PROCESSO NA CAIXA DE PESQUISA DO SEI PARA ACESSAR O PROCESSO
navegador.switch_to.default_content()
navegador.find_element(By.ID,'txtPesquisaRapida').send_keys(processo) 
elementos = navegador.find_element(By.ID,'txtPesquisaRapida')
elementos.send_keys(Keys.ENTER)    
#TEMPO DEFINIDO PARA QUE A PAGINA CARREGUE
time.sleep(1)

In [82]:
#PESQUISA NUMERO DO PROCESSO NA CAIXA DE PESQUISA DO SEI PARA ACESSAR O PROCESSO
navegador.switch_to.default_content()
navegador.find_element(By.ID,'txtPesquisaRapida').send_keys(processo) 
elementos = navegador.find_element(By.ID,'txtPesquisaRapida')
elementos.send_keys(Keys.ENTER)    
#TEMPO DEFINIDO PARA QUE A PAGINA CARREGUE
time.sleep(1)
#ENTRA NO FRAME QUE CONTÉM OS DOCUMENTOS DOS PROCESSOS
#A PAGINA DOS PROCESSOS SAO DIVIDIDOS EM FRAMES
navegador.switch_to.frame('ifrArvore')
#VERIFICA SE O PROCESSO JA FOI DESPACHADO, CASO TENHA SIDO ELE PULA O PROCESSO
#CONFERE SE EXISTE A DECLARACAO DE CONFORMIDADE OU UMA PASTA
if check_element_exists(By.PARTIAL_LINK_TEXT, "Declaração de Conformidade", navegador):
    navegador.find_element(By.PARTIAL_LINK_TEXT, "Declaração de Conformidade").click()
    time.sleep(0.3)
elif check_element_exists(By.XPATH, '//*[@id="spanPASTA1"]', navegador):
    navegador.find_element(By.XPATH, '//*[@id="spanPASTA1"]').click()
    time.sleep(1)
    navegador.find_element(By.PARTIAL_LINK_TEXT, "Declaração de Conformidade").click()
    time.sleep(0.3)

#CONFERE QUAL É O TIPO DE PROCESSO
if check_element_exists(By.PARTIAL_LINK_TEXT, "Declaração de Conformidade - Drone", navegador):
    tipo_processo = "Drone"
elif check_element_exists(By.PARTIAL_LINK_TEXT, "Declaração de Conformidade - Importado Uso Próprio", navegador):
    tipo_processo = "Importado"


In [83]:
#ESCREVER AS INFORMAÇÕES DO PROCESSO NO CONSOLE
#ACESSA O RECIBO PARA PEGAR NOME DO SOLICITANTE
navegador.find_element(By.PARTIAL_LINK_TEXT, 'Recibo Eletrônico').click()
navegador.switch_to.default_content()
#ENTRA NOS FRAMES QUE POSSUE O NOME DO SOLICITANTE
navegador.switch_to.frame('ifrVisualizacao')
navegador.switch_to.frame('ifrArvoreHtml')
#ARMAZENA NOME DO SOLICITANTE NA VARIAVEL
nomeSol = navegador.find_element(By.XPATH, '//*[@id="conteudo"]/table/tbody/tr[1]/td[2]').text
#RETORNA AO FRAME DEFAULT
navegador.switch_to.default_content()
#COLETAR DADOS DA DECLARAÇÃO DE CONFORMIDADE
#RETORNA AO FRAME QUE CONTÉM OS DOCUMENTOS
navegador.switch_to.frame('ifrArvore')

if tipo_processo == "Drone":
    #CLICA NA DECLARACAO DE CONFORMIDADE
    navegador.find_element(By.PARTIAL_LINK_TEXT, "Declaração de Conformidade - Drone").click()
    time.sleep(1)
elif tipo_processo == "Importado":
    #CLICA NA DECLARACAO DE CONFORMIDADE
    navegador.find_element(By.PARTIAL_LINK_TEXT, "Declaração de Conformidade - Importado Uso Próprio").click()
    time.sleep(1)

#RETORNA AO FRAME DEFAULT E ENTRA NOS FRAMES QUE CONTÉM A VISUALIZACAO DA DECLARACAO 
navegador.switch_to.default_content()
navegador.switch_to.frame('ifrVisualizacao')
navegador.switch_to.frame('ifrArvoreHtml')
#EXIBE INFORMACOES DO PROCESSO
print("--------------------------------------------------------------------------")
print('--------------------------------------------------------------------------')
print(f"Processo: {processo}")

#COLETAR TEXTO NOME DO INTERESSADO
if tipo_processo == "Drone":
    nome_interessado = navegador.find_element(By.XPATH, '/html/body/table[1]/tbody/tr/td').text
elif tipo_processo == "Importado":
    nome_interessado=navegador.find_element(By.XPATH, '/html/body/table[1]/tbody/tr/td/p').text

#VERIFICA SE O CAMPO ESTÁ VAZIO, CASO ESTEJA DEFINIRA CM NAO INFORMADO
if not nome_interessado.strip():
    nome_interessado = 'Não informado'
    print("NOME DO INTERESSADO NÃO INFORMADO")
#EXIBE NOMES DO INTERESSADO E SOLICITANTE
print("Interessado: ",nome_interessado)
print("Solicitante: ",nomeSol)
print('\n')

#CHECKBOX DE QUEM ESTÁ FAZENDO O PETICIONAMENTO
if tipo_processo == "Drone":
    checkbox=navegador.find_element(By.XPATH,'/html/body/table[2]/tbody/tr[1]/td[1]').text
    checkbox2=navegador.find_element(By.XPATH, '/html/body/table[2]/tbody/tr[2]/td[1]').text
elif tipo_processo == "Importado":
    checkbox=navegador.find_element('xpath','/html/body/table[2]/tbody/tr[1]/td[1]/p').text
    checkbox2=navegador.find_element(By.XPATH, '/html/body/table[2]/tbody/tr[2]/td[1]').text

#VERIFICA CAMPO DA CHECKBOX
#SE OS DOIS CAMPOS ESTIVEREM VAZIOS AVISA QUE NAO FOI INFORMADO QUEM ESTÁ FAZENDO O PETICIONAMENTO
if not checkbox.strip() and not checkbox2.strip():
    print("O usuário não informou quem está fazendo o peticionamento, confira os nomes do documento.")
elif not checkbox.strip():
    procuracao_eletronica = navegador.find_element(By.XPATH, '/html/body/table[2]/tbody/tr[2]/td[3]').text
    print("Peticionamento representando terceiro, Pessoa Jurídica ou Pessoa Física.", f"\nProcuração eletrônica: {procuracao_eletronica}.")
else:
    print("Peticionamento em interesse próprio como Pessoa Física.")
print('\n')

if tipo_processo == "Importado":
    #INFORMA O TIPO DE PRODUTO QUE ESTÁ SENDO TRATADO
    print("\nTipo de produto:")
    quantidade_linhasPro = len(navegador.find_elements(By.XPATH, '/html/body/table[3]/tbody/tr'))
    tipoPro = []
    for i in range(1, quantidade_linhasPro, 1):
        linhas = navegador.find_element(By.XPATH, f'/html/body/table[3]/tbody/tr[{i}]')
        celulas2 = linhas.find_elements(By.TAG_NAME,'td')
        produtos = [celula2.text for celula2 in celulas2]
        tipoPro.append(produtos)
    dfPro = pd.DataFrame(tipoPro)
    dfPro.columns = ['Tipo de Equipamento', 'Assinale com um X', 'Observação:']
    dfPro = dfPro.drop('Observação:', axis=1)
    dfPro = dfPro[1:]

    #EXIBE A TABELA DO TIPO DE PRODUTO
    data = dfPro.values.tolist()
    headers = dfPro.columns.tolist()
    print(tabulate(data, headers=headers, tablefmt='pretty'))
    print('\n')

#CONTA QUANTIDADE DE LINHAS DA TABELA SOBRE O PRODUTO
if tipo_processo == "Drone":
    quantidade_linhas = len(navegador.find_elements(By.XPATH, '/html/body/table[3]/tbody/tr'))

    #VERIFICA CAMPOS QUE CONTÉM AS INFORMAÇÕES SOBRE O PRODUTO
    #ARMAZENA OS MODELOS INFORMADOS PARA VERIFICAR SE ESTÃO NA PLANILHA DE DRONES CONFORMES
    modelos = []
    for i in range(1, quantidade_linhas, 1):
        linha = navegador.find_element('xpath', f'/html/body/table[3]/tbody/tr[{i}]')
        celulas = linha.find_elements(By.TAG_NAME,'td')
        modelos_produto = [celula.text for celula in celulas]
        modelos.append(modelos_produto)
elif tipo_processo == "Importado":
    quantidade_linhas = len(navegador.find_elements(By.XPATH, '/html/body/table[4]/tbody/tr'))

    #VERIFICA CAMPOS QUE CONTÉM AS INFORMAÇÕES SOBRE O PRODUTO
    modelos = []
    for i in range(1, quantidade_linhas, 1):
        linha = navegador.find_element('xpath', f'/html/body/table[4]/tbody/tr[{i}]')
        celulas = linha.find_elements(By.TAG_NAME,'td')
        modelos_produto = [celula.text for celula in celulas]
        modelos.append(modelos_produto)


#CRIA DATAFRAME PARA PRINTAR AS INFORMAÇÕES DO PRODUTO JÁ TRATADAS
df = pd.DataFrame(modelos)
if tipo_processo == "Drone":
    df.columns = ['Modelo', 'Nome Comercial', 'Número de Série (incluindo rádio controle e óculos)']
elif tipo_processo == "Importado":
    df.columns = ['Modelo', 'Nome Comercial', 'Número de Série']
df = df[1:]
df = df.replace(' ', np.nan)
df = df.dropna(how='all')
data = df.values.tolist()
headers = df.columns.tolist()
print(tabulate(data, headers=headers, tablefmt='pretty'))
print('\n')

--------------------------------------------------------------------------
--------------------------------------------------------------------------
Processo: ['53500.077392/2024-14']
Interessado:  RINALDO PINTO MEIRELES
Solicitante:  Rinaldo Pinto Meireles


Peticionamento em interesse próprio como Pessoa Física.



Tipo de produto:
+---------------------------+-------------------+
|    Tipo de Equipamento    | Assinale com um X |
+---------------------------+-------------------+
|          Drone*           |                   |
|        Radioamador        |         X         |
| Serviço Móvel Aeronáutico |                   |
|  Serviço Móvel Marítimo   |                   |
|      Estação Terrena      |                   |
+---------------------------+-------------------+


+--------+----------------+-----------------+
| Modelo | Nome Comercial | Número de Série |
+--------+----------------+-----------------+
| FT-60R |  YAESU FT-60R  |    OK220115     |
+--------+----------------+